In [41]:
import pandas as pd
import summary_functions as sf
import arcpy
from arcgis import GeoAccessor
from arcgis.features import FeatureLayer
import os
# set workspace
arcpy.env.workspace = "F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb"
workspace           = r"F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb"
workspace_folder    = r"F:\GIS\PROJECTS\ResearchAnalysis\Demographics"

year_data = 2022

## Import census data and lookup lists

In [35]:
def get_census_data(featureset):
    service_id = {
        'raw_data':'28',
        'summaries':'18',
        'enrollment':'32',
        'visitation':'33',
        'water_use':'34'
    }

    service_number = service_id.get(featureset) 
    service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/'+service_number

    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

In [36]:
census_data = get_census_data('raw_data')
tahoe_geography_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'
# Create a FeatureLayer object directly from the REST map service URL
feature_layer = FeatureLayer(tahoe_geography_url)

# Create a spatial DataFrame from the FeatureLayer
sdf = GeoAccessor.from_layer(feature_layer)

In [37]:
county_zone = {
    '061':'North Lake',
    '031':'North Lake',
    '005': 'South Lake',
    '017':'South Lake'
}
census_data['Zone'] = census_data['county'].map(county_zone)
#This list is based on input from Kira and Rachel
TDC_Variables = pd.read_csv("../Census_Category_Lists/TDC_categories.csv")

In [38]:
# This could be cool to do for all years so we could show change
block_group_data = census_data.loc[(census_data['sample_level']=='block group')&(census_data['year_sample']==year_data)]
# Map Bipoc, seniors, youth, 0 vehicle households (fillna for no categories)
tdc_data = block_group_data.loc[block_group_data['variable_code'].isin(TDC_Variables['variable_code'])]
#These will become the feature class column name
sort_order = ['TRPAID', 'county', 'tract', 'Total_Population',
'Total_Households',
'Total_Disability_Population',
'Age_Under_18',
'Age_18_to_65',
'Age_65_and_Over',
'With_Disability',
'Vehicle_Available_0',
'Vehicle_Available_1',
'Vehicle_Available_2',
'Vehicle_Available_3',
'Vehicle_Available_4',
'Vehicle_Available_5',
'White_Alone',
'BIPOC',
'Speak_English_Not_Well_Not_at_All'
]


In [39]:
# Grab the category assigning function
tdc_grouped_data = sf.categorize_values(tdc_data, "../Census_Category_Lists/TDC_categories.csv", 'Broad Category', "")

tdc_flat = tdc_grouped_data.pivot(index=['TRPAID', 'county', 'tract'], columns='variable_name', values='value')
tdc_flat = tdc_flat.reset_index()
tdc_flat = tdc_flat[sort_order]

['GEO_ID', 'state', 'county', 'tract', 'block_group', 'variable_category', 'year_sample', 'sample_level', 'dataset', 'census_geom_year', 'TRPAID', 'NEIGHBORHOOD', 'Data_Scale', 'Zone', 'Broad Category']


c:\Users\amcclary\Documents\GitHub\Demographics\scripts\summary_functions.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories = pd.read_csv(category_csv)


In [40]:
merged_df = pd.merge(sdf, tdc_flat, on='TRPAID', how='inner')
columns_drop=['GlobalID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'Shape.STArea()', 'Shape.STLength()']
merged_df = merged_df.drop(columns=columns_drop)

In [42]:
## Export spatial dataframes to feature class to use in Spatial join
merged_df.spatial.to_featureclass(os.path.join(workspace, "Tahoe_BlockGroup_2022_TDC_Values"), sanitize_columns=False)

'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\Tahoe_BlockGroup_2022_TDC_Values'